In [21]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [22]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

1.5.3
2.8.4


In [23]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time



In [24]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(6)

In [25]:
# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy

for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132, resolution=None, default=None, variable_name=['A.0_ID flood wave shape'], pff=False)
RealParameter('A.1_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.1_Bmax'], pff=False)
RealParameter('A.1_pfail', 0, 1, resolution=None, default=None, variable_name=['A.1_pfail'], pff=False)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.2_Bmax'], pff=False)
RealParameter('A.2_pfail', 0, 1, resolution=None, default=None, variable_name=['A.2_pfail'], pff=False)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.3_Bmax'], pff=False)
RealParameter('A.3_pfail', 0, 1, resolution=None, default=None, variable_name=['A.3_pfail'], pff=False)
CategoricalParameter('A.3_Brate', [0, 1, 2])
RealParameter('A.4_Bmax', 30, 350, resolution=N

In [26]:
# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

IntegerParameter('0_RfR 0', 0, 1, resolution=None, default=None, variable_name=['0_RfR 0'], pff=False)
IntegerParameter('1_RfR 0', 0, 1, resolution=None, default=None, variable_name=['1_RfR 0'], pff=False)
IntegerParameter('2_RfR 0', 0, 1, resolution=None, default=None, variable_name=['2_RfR 0'], pff=False)
IntegerParameter('3_RfR 0', 0, 1, resolution=None, default=None, variable_name=['3_RfR 0'], pff=False)
IntegerParameter('4_RfR 0', 0, 1, resolution=None, default=None, variable_name=['4_RfR 0'], pff=False)
IntegerParameter('EWS_DaysToThreat', 0, 4, resolution=None, default=None, variable_name=['EWS_DaysToThreat'], pff=False)
IntegerParameter('A.1_DikeIncrease 0', 0, 10, resolution=None, default=None, variable_name=['A.1_DikeIncrease 0'], pff=False)
IntegerParameter('A.2_DikeIncrease 0', 0, 10, resolution=None, default=None, variable_name=['A.2_DikeIncrease 0'], pff=False)
IntegerParameter('A.3_DikeIncrease 0', 0, 10, resolution=None, default=None, variable_name=['A.3_DikeIncrease 0'

In [27]:
# defining specific policies
# for example, policy 1 is about extra protection in upper boundary
# policy 2 is about extra protection in lower boundary
# policy 3 is extra protection in random locations


def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}

baseline_policies = [
    Policy(
        "Baseline",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 0,"2_RfR 0": 0, "3_RfR 0": 0, "4_RfR 0": 0,
               "A.1_DikeIncrease 0": 0,"A.2_DikeIncrease 0": 0,"A.3_DikeIncrease 0": 0,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':0
               }
        )
    )]

max_policies=[
    Policy(
        "DI max",
        **dict(
            get_do_nothing_dict(),
            **{"A.1_DikeIncrease 0": 5,"A.2_DikeIncrease 0": 5,"A.3_DikeIncrease 0": 5,"A.4_DikeIncrease 0": 5, "A.5_DikeIncrease 0": 5}
        )
    ),
        Policy(
        "RfR max",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "1_RfR 0": 1,"2_RfR 0": 1, "3_RfR 0": 1, "4_RfR 0": 1,
               "A.1_DikeIncrease 0": 0,"A.2_DikeIncrease 0": 0,"A.3_DikeIncrease 0": 0,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':0
               }
        )
    ),
        Policy(
        "EWS max",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 0, "2_RfR 0": 0, "3_RfR 0": 0, "4_RfR 0": 0,
               "A.1_DikeIncrease 0": 0,"A.2_DikeIncrease 0": 0,"A.3_DikeIncrease 0": 0,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':4
               }
        )
    ),
        Policy(
        "All max",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "1_RfR 0": 1, "2_RfR 0": 1, "3_RfR 0": 1, "4_RfR 0": 1,
               "A.1_DikeIncrease 0": 5,"A.2_DikeIncrease 0": 5,"A.3_DikeIncrease 0": 5,"A.4_DikeIncrease 0": 5, "A.5_DikeIncrease 0": 5,
               'EWS_DaysToThreat':4
               }
        )
    )]
    
risk_transfer=[
        Policy(
        "Di-Ge RfR-Ov max",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 0,"2_RfR 0": 0, "3_RfR 0": 1, "4_RfR 0": 1,
               "A.1_DikeIncrease 0": 5,"A.2_DikeIncrease 0": 5,"A.3_DikeIncrease 0": 5,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':0
               }
        )
    ),
        Policy(
        "RfR-Ge DI-Ov max",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 0,"2_RfR 0": 0, "3_RfR 0": 1, "4_RfR 0": 1,
               "A.1_DikeIncrease 0": 3,"A.2_DikeIncrease 0": 2,"A.3_DikeIncrease 0": 4,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':0
               }
        )
    )]

infrastr_policies=[
        Policy(
        "Comb Green",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "1_RfR 0": 1,"2_RfR 0": 0, "3_RfR 0": 1, "4_RfR 0": 1,
               "A.1_DikeIncrease 0": 0,"A.2_DikeIncrease 0": 0,"A.3_DikeIncrease 0": 4,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':0
               }
        )
    ),
        Policy(
        "Comb Grey",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 1,"2_RfR 0": 0, "3_RfR 0": 1, "4_RfR 0": 0,
               "A.1_DikeIncrease 0": 3,"A.2_DikeIncrease 0": 0,"A.3_DikeIncrease 0": 4,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 4,
               'EWS_DaysToThreat':0
               }
        )
    ),
        Policy(
        "Comb GreenEWS",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "1_RfR 0": 1,"2_RfR 0": 0, "3_RfR 0": 1, "4_RfR 0": 1,
               "A.1_DikeIncrease 0": 0,"A.2_DikeIncrease 0": 0,"A.3_DikeIncrease 0": 4,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':2
               }
        ))]

policy_old = [Policy(
        "Policy 1",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "0_RfR 1": 1, "0_RfR 2": 1, "A.1_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "Policy 2",
        **dict(
            get_do_nothing_dict(),
            **{"4_RfR 0": 1, "4_RfR 1": 1, "4_RfR 2": 1, "A.5_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "Policy 3",
        **dict(
            get_do_nothing_dict(),
            **{"1_RfR 0": 1, "2_RfR 1": 1, "3_RfR 2": 1, "A.3_DikeIncrease 0": 5}
        )
    )]



In [28]:
#Calling a function to convert csvs resulting from optimization to make a policies
from funs_policy_eval import create_optimization_policies

DR12_OP_v1=create_optimization_policies("OptimizationResults_DR12_v1.csv","DR12_v1_P",dike_model)


policies=baseline_policies+DR12_OP_v1+risk_transfer

In [29]:
# pass the policies list to EMA workbench experiment runs
n_scenarios = 100
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios, policies)

[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] performing 100 scenarios * 7 policies * 1 model(s) = 700 experiments
100%|████████████████████████████████████████| 700/700 [01:26<00:00,  8.05it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [30]:
experiments, outcomes = results
print(experiments.keys())
experiments.head()

Index(['A.0_ID flood wave shape', 'A.1_Bmax', 'A.1_Brate', 'A.1_pfail',
       'A.2_Bmax', 'A.2_Brate', 'A.2_pfail', 'A.3_Bmax', 'A.3_Brate',
       'A.3_pfail', 'A.4_Bmax', 'A.4_Brate', 'A.4_pfail', 'A.5_Bmax',
       'A.5_Brate', 'A.5_pfail', 'discount rate 0', '0_RfR 0', '1_RfR 0',
       '2_RfR 0', '3_RfR 0', '4_RfR 0', 'EWS_DaysToThreat',
       'A.1_DikeIncrease 0', 'A.2_DikeIncrease 0', 'A.3_DikeIncrease 0',
       'A.4_DikeIncrease 0', 'A.5_DikeIncrease 0', 'scenario', 'policy',
       'model'],
      dtype='object')


,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,4_RfR 0,EWS_DaysToThreat,A.1_DikeIncrease 0,A.2_DikeIncrease 0,A.3_DikeIncrease 0,A.4_DikeIncrease 0,A.5_DikeIncrease 0,scenario,policy,model
0,13,53.362937,1.5,0.747461,232.941734,1.0,0.925686,236.032299,1.5,0.990032,...,0,0,0,0,0,0,0,100,Baseline,dikesnet
1,90,271.274200,10.0,0.591856,164.797126,1.0,0.304664,294.336720,10.0,0.892881,...,0,0,0,0,0,0,0,101,Baseline,dikesnet
2,56,148.139167,1.5,0.972212,52.261908,1.0,0.880705,339.648986,1.5,0.852853,...,0,0,0,0,0,0,0,102,Baseline,dikesnet
3,88,214.655304,10.0,0.035033,57.251899,1.0,0.390115,223.060756,10.0,0.820847,...,0,0,0,0,0,0,0,103,Baseline,dikesnet
4,11,224.885374,1.0,0.713671,322.805197,1.0,0.105373,282.592272,10.0,0.967024,...,0,0,0,0,0,0,0,104,Baseline,dikesnet


In [31]:
from funs_viz import tidy_results

results_df = tidy_results(results,6,1)
results_df.to_csv("results/test_1.csv")

print(results_df.columns)
results_df.head()

Index(['A.0_ID flood wave shape', 'discount rate 0', 'EWS_DaysToThreat',
       'scenario', 'policy', 'model', 'RfR Total Costs',
       'Expected Evacuation Costs', 'Dike ring', 'Bmax', 'Brate', 'pfail',
       'RfR 0', 'DikeIncrease 0', 'Dike Investment Costs',
       'Expected Annual Damage', 'Expected Number of Deaths'],
      dtype='object')


,A.0_ID flood wave shape,discount rate 0,EWS_DaysToThreat,scenario,policy,model,RfR Total Costs,Expected Evacuation Costs,Dike ring,Bmax,Brate,pfail,RfR 0,DikeIncrease 0,Dike Investment Costs,Expected Annual Damage,Expected Number of Deaths
0,13,3.5,0,100,Baseline,dikesnet,0.0,0.0,A.1,53.362937,1.5,0.747461,0,0,0,4.229289e+06,0.003582
1,90,3.5,0,101,Baseline,dikesnet,0.0,0.0,A.1,271.274200,10.0,0.591856,0,0,0,2.733558e+07,0.022757
2,56,1.5,0,102,Baseline,dikesnet,0.0,0.0,A.1,148.139167,1.5,0.972212,0,0,0,0.000000e+00,0.000000
3,88,1.5,0,103,Baseline,dikesnet,0.0,0.0,A.1,214.655304,10.0,0.035033,0,0,0,1.285495e+09,0.566127
4,11,2.5,0,104,Baseline,dikesnet,0.0,0.0,A.1,224.885374,1.0,0.713671,0,0,0,1.046166e+07,0.006879
